<a href="https://colab.research.google.com/github/lovepreetmultani/Anti-Money-Laundering/blob/main/xgboost_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and Data load

In [1]:
!pip install torch torchvision
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=a63ff45927dc2ab95d057bc123cd9a5f95939ed0d70fda797bb74fbfd090f9b0
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [2]:
!pip install xgboost
import xgboost as xgb


In [3]:
import torch

# Check if CUDA (GPU support for PyTorch) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device("cpu")
    print('Using CPU')

Using CPU


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import time
from datetime import datetime as dt
from sklearn.feature_selection import  f_classif, SelectKBest
import matplotlib.pyplot as plt

In [5]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
fraud=pd.read_csv("drive/My Drive/Dissertation/Data-2-small.csv")

In [8]:
fraud.head()

,tx_fraud,tx_amount,s_pagerank,s_label,s_min_send_tx,s_min_receieve_tx,s_max_send_tx,s_max_recieve_tx,s_avg_send_tx,s_avg_recieve_tx,...,r_label,r_min_send_tx,r_min_receieve_tx,r_max_send_tx,r_max_recieve_tx,r_avg_send_tx,r_avg_recieve_tx,r_cnt_recieve_tx,r_cnt_send_tx,r_timestamp
0,True,2661.05,1.64276,52429084,90.98,45.89,2661.05,2956.72,174.60600,314.35879,...,52429264,91.04,46.12,109.53,2661.05,99.75712,851.45423,26,52,1509580800
1,True,2661.05,0.89618,37749079,90.19,45.72,2661.05,2956.72,207.55542,437.18238,...,52429264,91.04,46.12,109.53,2661.05,99.75712,851.45423,26,52,1509321600
2,True,2844.06,0.70773,34603485,90.30,48.52,2844.06,54.41,159.08872,52.09875,...,47186571,90.69,45.27,2559.65,2844.06,238.81746,200.33864,103,71,1499731200
3,True,2523.37,2.55504,49283515,91.41,45.03,2523.37,2803.73,270.59857,139.48263,...,34603040,90.58,45.65,109.96,2662.20,99.81943,781.37258,31,35,1485043200
4,True,2745.06,1.35612,55574890,90.66,45.34,2745.06,2718.14,245.75222,320.09300,...,57672098,90.34,45.72,2470.55,2745.06,314.67291,291.58139,79,55,1513987200


In [9]:
fraud.columns

Index(['tx_fraud', 'tx_amount', 's_pagerank', 's_label', 's_min_send_tx',
       's_min_receieve_tx', 's_max_send_tx', 's_max_recieve_tx',
       's_avg_send_tx', 's_avg_recieve_tx', 's_cnt_recieve_tx',
       's_cnt_send_tx', 's_timestamp', 'r_pagerank', 'r_label',
       'r_min_send_tx', 'r_min_receieve_tx', 'r_max_send_tx',
       'r_max_recieve_tx', 'r_avg_send_tx', 'r_avg_recieve_tx',
       'r_cnt_recieve_tx', 'r_cnt_send_tx', 'r_timestamp'],
      dtype='object')

## Data balacing

In [10]:
from imblearn.over_sampling import SMOTE

# ... Your existing imports and code for reading data ...

# Features and labels before SMOTE
features = fraud[['tx_amount', 's_pagerank', 's_label', 's_min_send_tx',
                 's_min_receieve_tx', 's_max_send_tx', 's_max_recieve_tx',
                 's_avg_send_tx', 's_avg_recieve_tx', 's_cnt_recieve_tx',
                 's_cnt_send_tx', 's_timestamp', 'r_pagerank', 'r_label',
                 'r_min_send_tx', 'r_min_receieve_tx', 'r_max_send_tx',
                 'r_max_recieve_tx', 'r_avg_send_tx', 'r_avg_recieve_tx',
                 'r_cnt_recieve_tx', 'r_cnt_send_tx', 'r_timestamp']]
labels = fraud['tx_fraud']

# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
features_resampled, labels_resampled = smote.fit_resample(features, labels)

# Replace your original 'fraud' DataFrame with the resampled data
fraud_resampled = features_resampled.copy()
fraud_resampled['tx_fraud'] = labels_resampled

In [11]:
fraud_resampled['tx_fraud'].value_counts()

True     9200
False    9200
Name: tx_fraud, dtype: int64

## Data preprocessing

In [12]:
import torch
from torch_geometric.data import Data


# Step 1: Map unique accounts to unique integer IDs
unique_accounts = pd.concat([fraud_resampled['s_label'], fraud_resampled['r_label']]).drop_duplicates()
account_to_id = {account: i for i, account in enumerate(unique_accounts)}

# Step 2: Create the edge_index tensor
source_node_indices = fraud_resampled['s_label'].map(account_to_id).values
target_node_indices = fraud_resampled['r_label'].map(account_to_id).values

edge_index = torch.tensor(
    [
        source_node_indices,  # Source nodes
        target_node_indices   # Target nodes
    ],
    dtype=torch.long
)

<ipython-input-12-170449250967>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  edge_index = torch.tensor(


In [13]:
from sklearn.preprocessing import StandardScaler

# Extract the features from the resampled data
features_to_scale = ['tx_amount', 's_pagerank', 's_label', 's_min_send_tx',
                     's_min_receieve_tx', 's_max_send_tx', 's_max_recieve_tx',
                     's_avg_send_tx', 's_avg_recieve_tx', 's_cnt_recieve_tx',
                     's_cnt_send_tx', 's_timestamp', 'r_pagerank', 'r_label',
                     'r_min_send_tx', 'r_min_receieve_tx', 'r_max_send_tx',
                     'r_max_recieve_tx', 'r_avg_send_tx', 'r_avg_recieve_tx',
                     'r_cnt_recieve_tx', 'r_cnt_send_tx', 'r_timestamp']

# Initialize the scaler
scaler = StandardScaler()

# Apply scaling to the features
fraud_data_scaled = fraud_resampled.copy()
fraud_data_scaled[features_to_scale] = scaler.fit_transform(fraud_resampled[features_to_scale])


# Use the scaled features in the resampled dataset
fraud_resampled_scaled = fraud_data_scaled.copy()

# Node features for the sender and receiver
sender_features_scaled = torch.tensor(fraud_resampled_scaled[['s_pagerank', 's_label', 's_min_send_tx',
                                  's_min_receieve_tx', 's_max_send_tx', 's_max_recieve_tx',
                                  's_avg_send_tx', 's_avg_recieve_tx', 's_cnt_recieve_tx',
                                  's_cnt_send_tx', 's_timestamp']].values, dtype=torch.float)

receiver_features_scaled = torch.tensor(fraud_resampled_scaled[['r_pagerank', 'r_label', 'r_min_send_tx',
                                    'r_min_receieve_tx', 'r_max_send_tx', 'r_max_recieve_tx',
                                    'r_avg_send_tx', 'r_avg_recieve_tx', 'r_cnt_recieve_tx',
                                    'r_cnt_send_tx', 'r_timestamp']].values, dtype=torch.float)

# Combining the sender and receiver features
x_scaled = torch.cat([sender_features_scaled, receiver_features_scaled], dim=1)

# Edge features
edge_attr_scaled = torch.tensor(fraud_resampled_scaled[['tx_amount']].values, dtype=torch.float)

# Create a PyTorch Geometric data object with scaled features
data_scaled = Data(x=x_scaled, edge_index=edge_index, edge_attr=edge_attr_scaled)


## XGBoost

Test train split

In [14]:
from sklearn.model_selection import train_test_split

# Initial train-test split
train_fraud, temp_fraud = train_test_split(fraud_resampled_scaled, test_size=0.4, random_state=42, stratify=fraud_resampled_scaled['tx_fraud'])

# Split the remaining 40% into validation and test sets
val_fraud, test_fraud = train_test_split(temp_fraud, test_size=0.5, random_state=42, stratify=temp_fraud['tx_fraud'])

In [15]:
X_train = fraud_resampled_scaled.loc[train_fraud.index, features_to_scale]
y_train = fraud_resampled_scaled.loc[train_fraud.index, 'tx_fraud']

X_val = fraud_resampled_scaled.loc[val_fraud.index, features_to_scale]
y_val = fraud_resampled_scaled.loc[val_fraud.index, 'tx_fraud']

X_test = fraud_resampled_scaled.loc[test_fraud.index, features_to_scale]
y_test = fraud_resampled_scaled.loc[test_fraud.index, 'tx_fraud']




In [16]:
!pip install explainerdashboard
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 93.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Train

In [18]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRFClassifier
from sklearn.metrics import precision_score
from pathlib import Path
pkl_dir = Path.cwd() / "pkls"

# Define the parameter grid for XGBoost
param_grid = {
    'n_estimators': [10],
    'max_depth': [3, 4],
    'learning_rate': [0.001],
    'gamma': [0],
    'min_child_weight': [1],
    'booster': ['gbtree']
}

# Initialize the XGBoost model
xgb_model = XGBRFClassifier(use_label_encoder=False, n_jobs=-1, max_depth= 5, min_child_weight=10, gamma=1, scale_pos_weight=99, max_delta_step=1, eval_metric='logloss')

# Set up the grid search with 5-fold cross validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           scoring='precision', cv=5, verbose=2, n_jobs=-1)

# Fit the grid search to the data
xgb_model.fit(X_train, y_train)

class_explainer = ClassifierExplainer(xgb_model, X_test, y_test,
                               labels=['Normal', 'Fraud'])
_ = ExplainerDashboard(class_explainer)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train and evaluate the best model
best_xgb = grid_search.best_estimator_

# Making predictions on the test set using the best XGBoost model
xgb_test_predictions = best_xgb.predict(X_test)

# Evaluate using Precision on the test set
xgb_test_precision = precision_score(y_test, xgb_test_predictions)
print(f"XGBoost Test Precision with best model: {xgb_test_precision}")

# Making predictions on the validation set using the best XGBoost model
xgb_val_predictions = best_xgb.predict(X_val)

# Evaluate using Precision on the validation set
xgb_val_precision = precision_score(y_val, xgb_val_predictions)
print(f"XGBoost Validation Precision with best model: {xgb_val_precision}")

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



Note: shap values for shap='kernel' normally get calculated against X_background, but paramater X_background=None, so setting X_background=shap.sample(X, 50)...
Generating self.shap_explainer = shap.KernelExplainer(model, X, link='identity')
Building ExplainerDashboard..
Detected google colab environment, setting mode='external'
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...


/usr/local/lib/python3.10/dist-packages/dash/dash.py:525: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



  0%|          | 0/3680 [00:00<?, ?it/s]

Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating pred_percentiles...
Calculating predictions...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...


AttributeError: ignored

In [ ]:
import fasttreeshap
shap_explainer = fasttreeshap.TreeExplainer(xgb_model)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score
x_result = confusion_matrix(y_test, xgb_test_predictions)
print("Confusion Matrix:")
print(x_result)
print('\n')
result1 = classification_report(y_test, xgb_test_predictions)
print("Classification Report:",)
print (result1)
print('\n')
x_result2 = accuracy_score(y_test,xgb_test_predictions)
print("Accuracy:",x_result2)
print('\n')
# calculate precision and recall scores
x_precision = precision_score(y_test, xgb_test_predictions)
x_recall = recall_score(y_test, xgb_test_predictions)
x_f1score = f1_score(y_test, xgb_test_predictions)

# print the results
print("Precision:", x_precision)
print("Recall:", x_recall)
print("f1 score:", x_f1score)

print(roc_auc_score(y_test, xgb_test_predictions))

fpr, tpr, _ = roc_curve(y_test, xgb_test_predictions)

plt.clf()
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
from xgboost import plot_importance, to_graphviz
fig = plt.figure(figsize = (5, 3))
ax = fig.add_subplot(111)

colours = plt.cm.Set1(np.linspace(0, 1, 9))

ax = plot_importance(xgb_model, height = 1, color = colours, grid = False, \
                     show_values = False, importance_type = 'cover', ax = ax);
for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(2)

ax.set_xlabel('importance score', size = 16);
ax.set_ylabel('features', size = 16);
ax.set_yticklabels(ax.get_yticklabels(), size = 12);
ax.set_title('Ordering of features by importance to the model learnt', size = 20);
to_graphviz(xgb_model)

## KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score

# Define the parameter grid for KNN
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Initialize the KNN model
knn_model = KNeighborsClassifier()

# Set up the grid search with 5-fold cross validation
grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid,
                           scoring='precision', cv=5, verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train and evaluate the best model
best_knn = grid_search.best_estimator_

# Making predictions on the test set using the best KNN model
knn_test_predictions = best_knn.predict(X_test)

# Evaluate using Precision on the test set
knn_test_precision = precision_score(y_test, knn_test_predictions)
print(f"KNN Test Precision with best model: {knn_test_precision}")

# Making predictions on the validation set using the best KNN model
knn_val_predictions = best_knn.predict(X_val)

# Evaluate using Precision on the validation set
knn_val_precision = precision_score(y_val, knn_val_predictions)
print(f"KNN Validation Precision with best model: {knn_val_precision}")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score
k_result = confusion_matrix(y_test, knn_test_predictions)
print("Confusion Matrix:")
print(k_result)
print('\n')
result1 = classification_report(y_test, knn_test_predictions)
print("Classification Report:",)
print (result1)
print('\n')
k_result2 = accuracy_score(y_test,knn_test_predictions)
print("Accuracy:",k_result2)
print('\n')
# calculate precision and recall scores
k_precision = precision_score(y_test, knn_test_predictions)
k_recall = recall_score(y_test, knn_test_predictions)
k_f1score = f1_score(y_test, knn_test_predictions)

# print the results
print("Precision:", k_precision)
print("Recall:", k_recall)
print("f1 score:", k_f1score)

print(roc_auc_score(y_test, knn_test_predictions))

fpr, tpr, _ = roc_curve(y_test, knn_test_predictions)

plt.clf()
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

##Decision Tree

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score

# Define the parameter grid for Decision Tree
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 3, 5, 7, 9, 11],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Set up the grid search with 5-fold cross validation
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid,
                           scoring='precision', cv=5, verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train and evaluate the best model
best_dt = grid_search.best_estimator_

# Making predictions on the validation set using the best Decision Tree model
dt_val_predictions = best_dt.predict(X_val)

# Evaluate using Precision on the validation set
dt_val_precision = precision_score(y_val, dt_val_predictions)
print(f"Decision Tree Validation Precision with best model: {dt_val_precision}")

# Making predictions on the test set using the best Decision Tree model
dt_test_predictions = best_dt.predict(X_test)

# Evaluate using Precision on the test set
dt_test_precision = precision_score(y_test, dt_test_predictions)
print(f"Decision Tree Test Precision with best model: {dt_test_precision}")


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Note: model_output=='probability', so assuming that raw shap output of DecisionTreeClassifier is in probability space...
Generating self.shap_explainer = shap.TreeExplainer(model)


AttributeError: ignored

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score
dt_result = confusion_matrix(y_test, dt_test_predictions)
print("Confusion Matrix:")
print(dt_result)
print('\n')
result1 = classification_report(y_test, dt_test_predictions)
print("Classification Report:",)
print (result1)
print('\n')
d_result2 = accuracy_score(y_test,dt_test_predictions)
print("Accuracy:",d_result2)
print('\n')
# calculate precision and recall scores
d_precision = precision_score(y_test, dt_test_predictions)
d_recall = recall_score(y_test, dt_test_predictions)
d_f1score = f1_score(y_test, dt_test_predictions)

# print the results
print("Precision:", d_precision)
print("Recall:", d_recall)
print("f1 score:", d_f1score)

print("ROC AUC Score",roc_auc_score(y_test, dt_test_predictions))

fpr, tpr, _ = roc_curve(y_test, dt_test_predictions)

plt.clf()
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

## RANDOM FOREST

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=10, random_state=42,max_depth=5,class_weight='balanced')

# Set up the grid search with 5-fold cross validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           scoring='precision', cv=5, verbose=2, n_jobs=-1)

# Fit the grid search to the data
rf_model.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train and evaluate the best model
best_rf = grid_search.best_estimator_

# Making predictions on the validation set using the best Random Forest model
rf_val_predictions = best_rf.predict(X_val)

# Evaluate using Precision on the validation set
rf_val_precision = precision_score(y_val, rf_val_predictions)
print(f"Random Forest Validation Precision with best model: {rf_val_precision}")

# Making predictions on the test set using the best Random Forest model
rf_test_predictions = best_rf.predict(X_test)

# Evaluate using Precision on the test set
rf_test_precision = precision_score(y_test, rf_test_predictions)
print(f"Random Forest Test Precision with best model: {rf_test_precision}")


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


KeyboardInterrupt: ignored

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=10, random_state=42,max_depth=5,class_weight='balanced')

# Set up the grid search with 5-fold cross validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid,
                           scoring='precision', cv=5, verbose=2, n_jobs=-1)

# Fit the grid search to the data
rf_model.fit(X_train, y_train)

class_explainer = ClassifierExplainer(rf_model, X_test, y_test,
                               labels=['Normal', 'Fraud'])
_ = ExplainerDashboard(class_explainer)

Detected RandomForestClassifier model: Changing class type to RandomForestClassifierExplainer...
Note: model_output=='probability', so assuming that raw shap output of RandomForestClassifier is in probability space...
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected google colab environment, setting mode='external'
Generating layout...
Calculating shap values...
Calculating prediction probabilities...
Calculating metrics...


/usr/local/lib/python3.10/dist-packages/dash/dash.py:525: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating shap interaction values... (this may take a while)
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating pred_percentiles...
Calculating predictions...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...


In [25]:
db = ExplainerDashboard(class_explainer,
                    whatif=False, # you can switch off tabs with bools
                    shap_interaction=False,
                    decision_trees=False)
db.run(port=8051)

Building ExplainerDashboard..
Detected google colab environment, setting mode='external'
Generating layout...
Calculating dependencies...


/usr/local/lib/python3.10/dist-packages/dash/dash.py:525: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://172.28.0.12:8051
You can terminate the dashboard with ExplainerDashboard.terminate(8051)


<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
rf_result = confusion_matrix(y_test, rf_test_predictions)
print("Confusion Matrix:")
print(rf_result)
print('\n')
result1 = classification_report(y_test, rf_test_predictions)
print("Classification Report:",)
print(result1)
print('\n')
rf_result2 = accuracy_score(y_test, rf_test_predictions)
print("Accuracy:", rf_result2)
print('\n')

# Calculate precision, recall, and F1-score
rf_precision = precision_score(y_test, rf_test_predictions)
rf_recall = recall_score(y_test, rf_test_predictions)
rf_f1score = f1_score(y_test, rf_test_predictions)

# Print the results
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("f1 score:", rf_f1score)
print(roc_auc_score(y_test, rf_test_predictions))

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, rf_test_predictions)
plt.clf()
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

## GNN


Model building

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [ ]:
class GNNModel(torch.nn.Module):
    def __init__(self, num_node_features):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(num_node_features, 32)  # Increased from 16 to 32
        self.conv2 = GCNConv(32, 16)  # New layer
        self.conv3 = GCNConv(16, 2)  # Output layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)  # New layer
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)


Test train split

In [ ]:
from sklearn.model_selection import train_test_split

# Initial train-test split
train_fraud, temp_fraud = train_test_split(fraud_resampled_scaled, test_size=0.4, random_state=42, stratify=fraud_resampled_scaled['tx_fraud'])

# Split the remaining 40% into validation and test sets
val_fraud, test_fraud = train_test_split(temp_fraud, test_size=0.5, random_state=42, stratify=temp_fraud['tx_fraud'])

# Initialize masks
train_mask = torch.zeros(len(fraud_resampled_scaled), dtype=torch.bool)
val_mask = torch.zeros(len(fraud_resampled_scaled), dtype=torch.bool)
test_mask = torch.zeros(len(fraud_resampled_scaled), dtype=torch.bool)

# Update masks based on the indices of the train, validation and test sets
train_mask[train_fraud.index] = True
val_mask[val_fraud.index] = True
test_mask[test_fraud.index] = True

# Update your data object
data_scaled.train_mask = train_mask
data_scaled.val_mask = val_mask
data_scaled.test_mask = test_mask
data_scaled.y = torch.tensor(fraud_resampled_scaled['tx_fraud'].values, dtype=torch.long)

Training

In [ ]:
# Number of node features
num_node_features = x_scaled.shape[1]

In [ ]:
import torch
from sklearn.metrics import precision_score

# Define the model, optimizer, and hyperparameters
def create_model_and_optimizer(learning_rate):
    GN_model = GNNModel(data_scaled.num_node_features)
    optimizer = torch.optim.Adam(GN_model.parameters(), lr=learning_rate)
    return GN_model, optimizer

# Grid of hyperparameters to search over
learning_rates = [0.001, 0.01, 0.1]
epochs_list = [50, 100, 200]

best_val_precision = 0
best_hyperparams = {}

# Manual grid search
for lr in learning_rates:
    for epochs in epochs_list:
        GN_model, optimizer = create_model_and_optimizer(lr)

        # Training loop with validation
        for epoch in range(epochs):
            GN_model.train()
            optimizer.zero_grad()
            out = GN_model(data_scaled)

            # Training loss
            train_loss = F.nll_loss(out[data_scaled.train_mask], data_scaled.y[data_scaled.train_mask])
            train_loss.backward()
            optimizer.step()

            # Calculate training precision
            _, train_pred = out[data_scaled.train_mask].max(dim=1)
            train_true = data_scaled.y[data_scaled.train_mask]
            train_precision = precision_score(train_true.cpu(), train_pred.cpu(), average='micro')

            # Validation
            GN_model.eval()  # Set the model to evaluation mode
            with torch.no_grad():  # Deactivate autograd engine to reduce memory usage and speed up computations
                val_loss = F.nll_loss(out[data_scaled.val_mask], data_scaled.y[data_scaled.val_mask])

                # Calculate validation precision
                _, val_pred = out[data_scaled.val_mask].max(dim=1)
                val_true = data_scaled.y[data_scaled.val_mask]
                val_precision = precision_score(val_true.cpu(), val_pred.cpu(), average='micro')

            # Print the metrics for every epoch
            print(f"LR: {lr}, Epochs: {epochs}, Epoch {epoch+1}, Training Loss: {train_loss.item()}, Validation Loss: {val_loss.item()}, Training Precision: {train_precision}, Validation Precision: {val_precision}")

        # At the end of training, check if the validation precision is the best so far
        if val_precision > best_val_precision:
            best_val_precision = val_precision
            best_hyperparams = {'learning_rate': lr, 'epochs': epochs}

print(f"Best Validation Precision: {best_val_precision}")
print(f"Best Hyperparameters: {best_hyperparams}")


In [ ]:
# Testing and Evaluation
GN_model.eval()
_, pred = out.max(dim=1)
correct = float(pred[data_scaled.test_mask].eq(data_scaled.y[data_scaled.test_mask]).sum().item())
acc = correct / data_scaled.test_mask.sum().item()
print(f"Test Accuracy: {acc}")

from sklearn.metrics import precision_score


# Testing and Evaluation
GN_model.eval()
_, pred = out.max(dim=1)

# Extract ground truth and predictions for test set
true_labels = data_scaled.y[data_scaled.test_mask].cpu().numpy()
predicted_labels = pred[data_scaled.test_mask].cpu().numpy()

In [ ]:
from sklearn.metrics import f1_score
g_result = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(g_result)
print('\n')
result1 = classification_report(true_labels, predicted_labels)
print("Classification Report:",)
print (result1)
print('\n')
g_result2 = accuracy_score(true_labels,predicted_labels)
print("Accuracy:",g_result2)
print('\n')
# calculate precision and recall scores
g_precision = precision_score(true_labels, predicted_labels)
g_recall = recall_score(true_labels, predicted_labels)
g_f1score = f1_score(true_labels, predicted_labels)

# print the results
print("Precision:", g_precision)
print("Recall:", g_recall)
print("f1 score:", g_f1score)

print(roc_auc_score(true_labels, predicted_labels))

fpr, tpr, _ = roc_curve(true_labels, predicted_labels)

plt.clf()
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the models and their performance metrics
models = ['KNN', 'XGBRFClassifier', 'Decision Tree', 'Graph Convolution Network']
accuracy = [k_result2, x_result2, d_result2, g_result2]
precision = [k_precision, x_precision, d_precision, g_precision]
recall = [k_recall, x_recall, d_recall, g_recall]
f1_score = [k_f1score, x_f1score, d_f1score, g_f1score]

# Create a single plot for all metrics using grouped bar chart
fig, ax = plt.subplots(figsize=(10, 5))
fig.suptitle('Model Performance Comparison Across Metrics')

# Set the positions and width for the bars
pos = np.arange(len(models))
bar_width = 0.2

# Plot bars for each metric
rects1 = ax.bar(pos - bar_width*1.5, accuracy, bar_width, label='Accuracy', color='blue')
rects2 = ax.bar(pos - bar_width/2, precision, bar_width, label='Precision', color='green')
rects3 = ax.bar(pos + bar_width/2, recall, bar_width, label='Recall', color='red')
rects4 = ax.bar(pos + bar_width*1.5, f1_score, bar_width, label='F1 Score', color='purple')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Models')
ax.set_ylabel('Metrics Value')
ax.set_xticks(pos)
ax.set_xticklabels(models)
ax.set_ylim([0.7, 1.0])
ax.legend()

# Label with specially formatted floats
ax.bar_label(rects1, fmt='%.3f')
ax.bar_label(rects2, fmt='%.3f')
ax.bar_label(rects3, fmt='%.3f')
ax.bar_label(rects4, fmt='%.3f')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
pip install torchviz

In [ ]:
import torch
from torchviz import make_dot
from IPython.display import Image
import os

# Initialize the model
GN_model = GNNModel(data_scaled.num_node_features)

# Generate dummy output using a sample from your dataset
dummy_output = GN_model(data_scaled)

# Generate the visualization
dot = make_dot(dummy_output, params=dict(GN_model.named_parameters()))

# Save and display
dot.format = 'png'
dot.render(filename='architecture', directory='./', cleanup=True)
display(Image(filename='architecture.png'))


In [ ]:
!pip install hiddenlayer


## Comparisions

In [ ]:
# Define a function to plot the combined ROC curves
def plot_combined_roc(y_test, *models):
    plt.figure(figsize=(10, 8))

    for model_predictions, model_name in models:
        fpr, tpr, _ = roc_curve(y_test, model_predictions)
        plt.plot(fpr, tpr, label=f'{model_name}')

    # Formatting
    plt.plot([0, 1], [0, 1], 'k--')  # diagonal 45 degree line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('Combined ROC Curve')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.2)
    plt.show()

# Call the function after predictions for each model are available
plot_combined_roc(y_test,
                  (xgb_test_predictions, "XGBoost"),
                  (knn_test_predictions, "KNN"),
                  (dt_test_predictions, "Decision Tree"),
                  (predicted_labels, "GNN"))  # assuming 'predicted_labels' is the GNN's predictions


## MPNN

In [ ]:
from torch_geometric.nn import MessagePassing

In [ ]:
class MPNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MPNNLayer, self).__init__(aggr='mean')  # "Mean" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, in_channels]
        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        return x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]
        return aggr_out

In [ ]:
class MPNNModel(torch.nn.Module):
    def __init__(self, num_node_features):
        super(MPNNModel, self).__init__()
        self.layer1 = MPNNLayer(num_node_features, 64)  # Increase to 64 nodes
        self.layer2 = MPNNLayer(64, 32)  # New layer
        self.layer3 = MPNNLayer(32, 32)  # New layer
        self.layer4 = MPNNLayer(32, 16)  # Reduce to 16 nodes
        self.layer5 = MPNNLayer(16, 2)   # Output layer for binary classification

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.layer1(x, edge_index))
        x = F.relu(self.layer2(x, edge_index))
        x = F.relu(self.layer3(x, edge_index))
        x = F.relu(self.layer4(x, edge_index))
        x = self.layer5(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
# Initialize the model and optimizer
MPNN_model = MPNNModel(data_scaled.num_node_features)
optimizer = torch.optim.Adam(MPNN_model.parameters(), lr=0.001)
# Initialize the SGD optimizer with momentum

# Lists to hold loss and precision values for plotting or analysis
train_losses = []
val_losses = []
train_precisions = []
val_precisions = []

# Training loop with validation
for epoch in range(200):
    MPNN_model.train()
    optimizer.zero_grad()
    out = MPNN_model(data_scaled)

    # Training loss
    train_loss = F.nll_loss(out[data_scaled.train_mask], data_scaled.y[data_scaled.train_mask])
    train_loss.backward()
    optimizer.step()

    # Calculate training precision
    _, train_pred = out[data_scaled.train_mask].max(dim=1)
    train_true = data_scaled.y[data_scaled.train_mask]
    train_precision = precision_score(train_true.cpu(), train_pred.cpu(), average='micro')

    # Validation
    MPNN_model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Deactivate autograd engine to reduce memory usage and speed up computations
        val_loss = F.nll_loss(out[data_scaled.val_mask], data_scaled.y[data_scaled.val_mask])

        # Calculate validation precision
        _, val_pred = out[data_scaled.val_mask].max(dim=1)
        val_true = data_scaled.y[data_scaled.val_mask]
        val_precision = precision_score(val_true.cpu(), val_pred.cpu(), average='micro')

    # Print the metrics
    print(f"Epoch {epoch+1}, Training Loss: {train_loss.item()}, Validation Loss: {val_loss.item()}, Training Precision: {train_precision}, Validation Precision: {val_precision}")

In [ ]:
!pip install explaineddashboard